In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git # importer le modèle
!pip install nibabel
!pip install scipy
!pip install matplotlib

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-negabwf6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-negabwf6
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=62bd91e24318f02c08ba2dd4f8c015f530c39d1d3864288d1e3ac1a461ed33c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-_4625fcd/wheels/29/82/ff/04e2be9805a1cb48bec0b85b5a6da6b63f647645750a0e42d4
Successfully built segment_anything


In [2]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth #importer les poids

--2025-11-17 09:24:22--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.37.111, 13.35.37.123, 13.35.37.84, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.37.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M   162MB/s    in 2.2s    

2025-11-17 09:24:24 (162 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [3]:
from google.colab import drive # connecter au drive pour avoir acces aux IRM
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
from pathlib import Path
import numpy as np
import nibabel as nib
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from scipy.spatial.distance import directed_hausdorff
import matplotlib.pyplot as plt

In [5]:
torch.cuda.is_available() # vérifier qu'on puisse utiliser un gpu

True

In [6]:
def load_nii(path): # charger les IRM
    img = nib.load(str(path))
    arr = img.get_fdata(dtype=np.float32)
    arr = np.transpose(arr, (2,1,0))  # (Z,Y,X)
    return arr, img

def zscore(x, eps=1e-6): # normaliser les images
    m, s = x.mean(), x.std()
    return (x - m) / (s + eps)

def dice_score(y_true, y_pred): # calculer indice Dice
    intersection = np.sum(y_true * y_pred)
    return 2. * intersection / (np.sum(y_true) + np.sum(y_pred) + 1e-6)

def iou_score(y_true, y_pred): # calculer indice IoU
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return intersection / (union + 1e-6)

In [7]:
ROOT = Path("/content/drive/MyDrive/projet_ML_CS/train")
OUT_DIR = Path("/content/drive/MyDrive/projet_ML_CS/sam_predictions")
OUT_DIR.mkdir(exist_ok=True, parents=True)

SAM_MODEL = "vit_b"
SAM_CKPT = "/content/drive/MyDrive/projet_ML_CS/sam_vit_b_01ec64.pth"
DEVICE = "cuda"

In [8]:
sam = sam_model_registry[SAM_MODEL](checkpoint=SAM_CKPT)
sam.to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)

In [9]:
patient_dir = sorted(ROOT.glob("*"))[0] # test sur le premier patient
print(f"Patient : {patient_dir.name}")

flair_path = next(patient_dir.glob("*flair*.nii*"), None)
seg_path = next(patient_dir.glob("*seg*.nii*"), None)

flair, ref_img = load_nii(flair_path)
seg, _ = load_nii(seg_path)
flair = zscore(flair)
seg = (seg > 0).astype(np.uint8)

pred_mask = np.zeros_like(flair, dtype=np.uint8)

# --- SAM slice par slice
for z in range(flair.shape[0]):
    slice2d = flair[z]

    # Ajustement contraste et conversion en uint8
    slice2d_uint8 = ((slice2d - slice2d.min()) / (slice2d.max() - slice2d.min() + 1e-6) * 255).astype(np.uint8)

    # Passage en 3 channels : obligatoire pour SAM
    slice2d_3c = np.stack([slice2d_uint8]*3, axis=-1)

    masks = mask_generator.generate(slice2d_3c)

    print(f"Slice {z} : {len(masks)} masques générés")

    if masks:
        largest = max(masks, key=lambda x: x['segmentation'].sum())
        pred_mask[z] = largest['segmentation'].astype(np.uint8)

        # Affichage pour debug
        plt.figure(figsize=(6,6))
        plt.imshow(slice2d_uint8, cmap='gray')
        plt.imshow(largest['segmentation'], alpha=0.5)
        plt.title(f"Slice {z} - masque superposé")
        plt.show()

# --- Sauvegarde masque
out_path = OUT_DIR / f"{patient_dir.name}_sam_mask.nii.gz"
nib.save(nib.Nifti1Image(np.transpose(pred_mask, (2,1,0)), affine=ref_img.affine), str(out_path))
print(f"Masque sauvegardé : {out_path}")

# --- Calcul des scores
dice = dice_score(seg, pred_mask)
iou = iou_score(seg, pred_mask)
print(f"Dice={dice:.3f}, IoU={iou:.3f}")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
results = []

# Boucle sur tous les patients
for patient_dir in sorted(ROOT.glob("*")):
    print(f"\n=== Traitement du patient : {patient_dir.name} ===")

    flair_path = next(patient_dir.glob("*flair*.nii*"), None)
    seg_path = next(patient_dir.glob("*seg*.nii*"), None)

    if not flair_path or not seg_path:
        print(f" Données manquantes pour {patient_dir.name}", flush=True)
        continue

    # --- Charger les volumes
    flair, ref_img = load_nii(flair_path)
    seg, _ = load_nii(seg_path)

    print(f"Flair shape : {flair.shape}, min/max : {flair.min():.3f}/{flair.max():.3f}")
    print(f"Seg shape : {seg.shape}, min/max : {seg.min():.3f}/{seg.max():.3f}")

    flair = zscore(flair)
    seg = (seg > 0).astype(np.uint8)
    pred_mask = np.zeros_like(flair, dtype=np.uint8)

    # --- SAM slice par slice
    for z in range(flair.shape[0]):
        slice2d = flair[z]

        # Conversion en uint8 avec contraste
        slice2d_uint8 = ((slice2d - slice2d.min()) / (slice2d.max() - slice2d.min() + 1e-6) * 255).astype(np.uint8)

        # Passage en 3 channels
        slice2d_3c = np.stack([slice2d_uint8]*3, axis=-1)

        masks = mask_generator.generate(slice2d_3c)

        if masks:
            # Conserver le plus grand masque
            largest = max(masks, key=lambda x: x['segmentation'].sum())
            pred_mask[z] = largest['segmentation'].astype(np.uint8)

    # --- Sauvegarde masque
    out_path = OUT_DIR / f"{patient_dir.name}_sam_mask.nii.gz"
    nib.save(nib.Nifti1Image(np.transpose(pred_mask, (2,1,0)), affine=ref_img.affine), str(out_path))
    print(f"Masque sauvegardé : {out_path}")

    # --- Scores
    dice = dice_score(seg, pred_mask)
    iou = iou_score(seg, pred_mask)
    print(f"{patient_dir.name} : Dice={dice:.3f}, IoU={iou:.3f}")
    results.append({"patient": patient_dir.name, "dice": dice, "iou": iou})

# --- Résumé global
if results:
    mean_dice = np.mean([r["dice"] for r in results])
    mean_iou = np.mean([r["iou"] for r in results])
    print(f"\nDice moyen : {mean_dice:.3f}, IoU moyen : {mean_iou:.3f}")
else:
    print(" Aucun résultat à calculer", flush=True)